In [1]:
#This project looks at the availability of different types of childcare in Toronto in 2021.
#I lightly clean the data and then focus on the parts which may yield interesting insights.
#I was looking to craete a summary of snapshot of childcare availability in Toronto while also looking more deeply at differences between age groups, subsidies, and geography.
The data is then visualized in two dashboards I created in Tableau.
The first dashboard provides a snapshot of Toronto childcare in January 2022 and can be found at https://public.tableau.com/app/profile/roman.khazin/viz/TorontoChildcare/TorontoChildcareSummary.
The second dashboard provides a more detailed picture, breaking down the data by age category. It can be found here: https://public.tableau.com/app/profile/roman.khazin/viz/TorontoChildcareDetailed/Dashboard1
#The data for this project is avaialable on the City of Toronto Open Data Portal at 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Importing Toronto childcare centre data and postal code population data
#Column meanings in childcare_centres:
#IGSPACE - infant spaces (0-18 months)
#TGSPACE - toddler spaces (18-30 months)
#PGSPACE - preschoolers spaces (30 months - grade 1)
#KGSPACE - kindergarten spaces (kids in full-day kindergarten)
#SGSPACE - school spaces (grade 1 and up)
#TOTSPACE - total spaces for all age groups

childcare_centres = pd.read_csv('C:\Data Analytics\Projects\Toronto Child Care\child_care_centres.csv')
postal_codes = pd.read_csv('C:\Python\Toronto Housing\postal_codes_population.csv')
print (childcare_centres.head())

   _id  LOC_ID                                           LOC_NAME  \
0    1    1013              Lakeshore Community Child Care Centre   
1    2    1014            Alternative Primary School Parent Group   
2    3    1015     Cardinal Leger Child Care Centre (Scarborough)   
3    4    1016  George Brown - Richmond Adelaide Childcare Centre   
4    5    1017  Woodland Nursery School (Warden Woods Communit...   

             AUSPICE            ADDRESS    PCODE  ward               PHONE  \
0  Non Profit Agency     101 SEVENTH ST  M8V 3B5     3      (416) 394-7601   
1  Non Profit Agency    1100 SPADINA RD  M5N 2M6     8      (416) 322-5385   
2  Non Profit Agency     600 MORRISH RD  M1C 4Y1    25      (416) 287-0578   
3  Non Profit Agency  130 ADELAIDE ST W  M5H 3P5    10      (416) 415-2453   
4  Non Profit Agency    1 FIRVALLEY CRT  M1L 1N8    20  (416) 694-1138x157   

                    bldg_type                         BLDGNAME  IGSPACE  \
0    Public Elementary School     Seventh

In [3]:
#Checking for duplicates in the LOC_ID column

print (childcare_centres[childcare_centres.duplicated(subset=['LOC_ID'])])

Empty DataFrame
Columns: [_id, LOC_ID, LOC_NAME, AUSPICE, ADDRESS, PCODE, ward, PHONE, bldg_type, BLDGNAME, IGSPACE, TGSPACE, PGSPACE, KGSPACE, SGSPACE, TOTSPACE, subsidy, run_date, geometry]
Index: []


In [4]:
#Removing irrelevant columns

childcare_centres = childcare_centres.drop(columns=['ADDRESS', 'PHONE', 'run_date', 'geometry'])
print (childcare_centres.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1034 entries, 0 to 1033
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   _id        1034 non-null   int64  
 1   LOC_ID     1034 non-null   int64  
 2   LOC_NAME   1034 non-null   object 
 3   AUSPICE    1034 non-null   object 
 4   PCODE      1034 non-null   object 
 5   ward       1034 non-null   int64  
 6   bldg_type  1034 non-null   object 
 7   BLDGNAME   716 non-null    object 
 8   IGSPACE    1033 non-null   float64
 9   TGSPACE    1033 non-null   float64
 10  PGSPACE    1033 non-null   float64
 11  KGSPACE    1033 non-null   float64
 12  SGSPACE    1033 non-null   float64
 13  TOTSPACE   1033 non-null   float64
 14  subsidy    1034 non-null   object 
dtypes: float64(6), int64(3), object(6)
memory usage: 121.3+ KB
None


In [5]:
#Converting the number of spaces columns into integer format instead of a float and filling in empty values with 0

childcare_centres = childcare_centres.fillna({'IGSPACE':'0', 'TGSPACE':'0', 'PGSPACE':'0', 'KGSPACE':'0', 'SGSPACE':'0', 'TOTSPACE':'0'})
childcare_centres = childcare_centres.astype({'IGSPACE':'int64', 'TGSPACE':'int64', 'PGSPACE':'int64', 'KGSPACE':'int64', 'SGSPACE':'int64', 'TOTSPACE':'int64'})
print (childcare_centres.head())

   _id  LOC_ID                                           LOC_NAME  \
0    1    1013              Lakeshore Community Child Care Centre   
1    2    1014            Alternative Primary School Parent Group   
2    3    1015     Cardinal Leger Child Care Centre (Scarborough)   
3    4    1016  George Brown - Richmond Adelaide Childcare Centre   
4    5    1017  Woodland Nursery School (Warden Woods Communit...   

             AUSPICE    PCODE  ward                   bldg_type  \
0  Non Profit Agency  M8V 3B5     3    Public Elementary School   
1  Non Profit Agency  M5N 2M6     8    Public Elementary School   
2  Non Profit Agency  M1C 4Y1    25  Catholic Elementary School   
3  Non Profit Agency  M5H 3P5    10                       Other   
4  Non Profit Agency  M1L 1N8    20         High Rise Apartment   

                          BLDGNAME  IGSPACE  TGSPACE  PGSPACE  KGSPACE  \
0     Seventh Street Public School        0       20       32       52   
1  North Preparatory Public School

In [6]:
#Counting the number of spaces by the type of auspice

auspices = childcare_centres.groupby('AUSPICE').TOTSPACE.sum()
auspices.to_csv('C:\Data Analytics\Projects\Toronto Child Care\childcare_auspices.csv')
print (auspices)

AUSPICE
Commercial Agency                18653
Non Profit Agency                58228
Public (City Operated) Agency     2027
Name: TOTSPACE, dtype: int64


In [7]:
#Comparing the number of spaces that are subsidized vs unsubsidized

subsidies = childcare_centres.groupby('subsidy').TOTSPACE.sum()
subsidies.to_csv('C:\Data Analytics\Projects\Toronto Child Care\childcare_subsidies.csv')
print (subsidies)

subsidy
N    17329
Y    61579
Name: TOTSPACE, dtype: int64


In [8]:
#Selecting columns for a more detailed analysis by auspice

auspice_detail = childcare_centres[['AUSPICE','IGSPACE', 'TGSPACE', 'PGSPACE', 'KGSPACE', 'SGSPACE']]
print (auspice_detail.head())

             AUSPICE  IGSPACE  TGSPACE  PGSPACE  KGSPACE  SGSPACE
0  Non Profit Agency        0       20       32       52       60
1  Non Profit Agency        0        0       12       26       45
2  Non Profit Agency        0       10       16       26       50
3  Non Profit Agency       10       15       40        0        0
4  Non Profit Agency        0       10       16        0        0


In [9]:
#Counting the number of spaces by auspice, broken down by age group

auspice_detail_grouped = auspice_detail.groupby('AUSPICE').sum()
auspice_detail_grouped.to_csv('C:\Data Analytics\Projects\Toronto Child Care\childcare_auspices_detailed.csv')
print (auspice_detail_grouped)

                               IGSPACE  TGSPACE  PGSPACE  KGSPACE  SGSPACE
AUSPICE                                                                   
Commercial Agency                 1479     4390     9296     1266     2222
Non Profit Agency                 2138     6218    14175    13808    21889
Public (City Operated) Agency      372      545      920       85      105


In [10]:
#Pivoting the previous table for greater usability and potential to create different visualizations

auspice_detail_pivot = pd.pivot_table(auspice_detail_grouped, columns='AUSPICE')
auspice_detail_pivot.to_csv('C:\Data Analytics\Projects\Toronto Child Care\childcare_auspices_detailed_pivot.csv')
print (auspice_detail_pivot)

AUSPICE  Commercial Agency  Non Profit Agency  Public (City Operated) Agency
IGSPACE               1479               2138                            372
KGSPACE               1266              13808                             85
PGSPACE               9296              14175                            920
SGSPACE               2222              21889                            105
TGSPACE               4390               6218                            545


In [11]:
#Selecting and grouping the number of spaces by subsidy, broken down by age group

subsidy_detail = childcare_centres[['subsidy','IGSPACE', 'TGSPACE', 'PGSPACE', 'KGSPACE', 'SGSPACE']]
subsidy_detail_grouped = subsidy_detail.groupby('subsidy').sum()
subsidy_detail_grouped.to_csv('C:\Data Analytics\Projects\Toronto Child Care\childcare_subsidy_detailed.csv')
print (subsidy_detail_grouped)

         IGSPACE  TGSPACE  PGSPACE  KGSPACE  SGSPACE
subsidy                                             
N           1058     4166     9501     1133     1471
Y           2931     6987    14890    14026    22745


In [12]:
#Pivoting the previous table

subsidy_detail_pivot = pd.pivot_table(subsidy_detail_grouped, columns='subsidy')
print (subsidy_detail_pivot)

subsidy     N      Y
IGSPACE  1058   2931
KGSPACE  1133  14026
PGSPACE  9501  14890
SGSPACE  1471  22745
TGSPACE  4166   6987


In [13]:
#Calculating the percentage of spaces that are subsidized in each age group

subsidy_detail_pivot['pct_subsidized'] = subsidy_detail_pivot['Y']/(subsidy_detail_pivot['Y']+subsidy_detail_pivot['N'])*100
subsidy_detail_pivot['pct_subsidized'] = subsidy_detail_pivot.pct_subsidized.round(1)
subsidy_detail_pivot.to_csv('C:\Data Analytics\Projects\Toronto Child Care\childcare_subsidy_detailed_pivot.csv')
print (subsidy_detail_pivot)

subsidy     N      Y  pct_subsidized
IGSPACE  1058   2931            73.5
KGSPACE  1133  14026            92.5
PGSPACE  9501  14890            61.0
SGSPACE  1471  22745            93.9
TGSPACE  4166   6987            62.6


In [14]:
#Calculating the number of spaces in each city ward

spaces_by_ward = childcare_centres.groupby('ward').TOTSPACE.sum()
print (spaces_by_ward)

ward
1     2112
2     3439
3     5042
4     4312
5     2290
6     3390
7     2508
8     4073
9     3145
10    2987
11    3461
12    3697
13    2090
14    4851
15    3839
16    3467
17    3683
18    2452
19    4482
20    2451
21    2169
22    2403
23    1935
24    2781
25    1849
Name: TOTSPACE, dtype: int64


In [15]:
#Calculating the number of spots by postal code, taken as the first letter and digit 

childcare_centres['PCODE'] = childcare_centres.PCODE.str.extract('(M.)')
spaces_by_postcode = childcare_centres.groupby('PCODE').TOTSPACE.sum().reset_index()
print (spaces_by_postcode)

  PCODE  TOTSPACE
0    M1     13588
1    M2      7477
2    M3      7958
3    M4     16089
4    M5      8015
5    M6     14008
6    M7        88
7    M8      4593
8    M9      7092


In [16]:
#Removing the M7 postal code row as it is a legislative building and irrelevant to the analysis

spaces_by_postcode = spaces_by_postcode.drop(spaces_by_postcode.index[6])
print (spaces_by_postcode)

  PCODE  TOTSPACE
0    M1     13588
1    M2      7477
2    M3      7958
3    M4     16089
4    M5      8015
5    M6     14008
7    M8      4593
8    M9      7092


In [17]:
#Looking at the population by postal code dataset

print (postal_codes.head())

  Geographic_code Geographic name      Province or territory  \
0               1          Canada                        NaN   
1             A0A             A0A  Newfoundland and Labrador   
2             A0B             A0B  Newfoundland and Labrador   
3             A0C             A0C  Newfoundland and Labrador   
4             A0E             A0E  Newfoundland and Labrador   

  Incompletely enumerated Indian reserves and Indian settlements, 2016  \
0                                                  T                     
1                                                NaN                     
2                                                NaN                     
3                                                NaN                     
4                                                NaN                     

   Population  Total_private_dwellings  \
0  35151728.0               15412443.0   
1     46587.0                  26155.0   
2     19792.0                  13658.0   
3 

In [18]:
#Calculating the population by postal code

postal_codes['Geographic_code'] = postal_codes.Geographic_code.str.extract('(M.)')
postcode_population = postal_codes.groupby('Geographic_code').Population.sum().reset_index()
print (postcode_population)

  Geographic_code  Population
0              M1    634047.0
1              M2    275211.0
2              M3    238916.0
3              M4    436223.0
4              M5    261749.0
5              M6    461368.0
6              M7        20.0
7              M8    107773.0
8              M9    316832.0
9              MA         0.0


In [19]:
#Converting the population column to an integer type from a float and dropping the M7 and MA postal codes

postcode_population = postcode_population.astype({'Population':'int64'})
postcode_population = postcode_population.drop(postcode_population.index[[6, 9]])
print (postcode_population)

  Geographic_code  Population
0              M1      634047
1              M2      275211
2              M3      238916
3              M4      436223
4              M5      261749
5              M6      461368
7              M8      107773
8              M9      316832


In [20]:
#Joining the postal code population and childcare spaces tables

all_postcode = pd.merge(spaces_by_postcode, postcode_population, left_on='PCODE', right_on='Geographic_code')
print (all_postcode)

  PCODE  TOTSPACE Geographic_code  Population
0    M1     13588              M1      634047
1    M2      7477              M2      275211
2    M3      7958              M3      238916
3    M4     16089              M4      436223
4    M5      8015              M5      261749
5    M6     14008              M6      461368
6    M8      4593              M8      107773
7    M9      7092              M9      316832


In [21]:
#Cleaning up the previous table and calculating the number of spaces per 100k residents by postal code

all_postcode = all_postcode.drop(columns=['Geographic_code'])
all_postcode['spaces_per_100k'] = all_postcode['TOTSPACE']/all_postcode['Population']*100000
all_postcode = all_postcode.astype({'spaces_per_100k':'int64'})
all_postcode.to_csv('C:\Data Analytics\Projects\Toronto Child Care\childcare_postcodes.csv')
print (all_postcode)

  PCODE  TOTSPACE  Population  spaces_per_100k
0    M1     13588      634047             2143
1    M2      7477      275211             2716
2    M3      7958      238916             3330
3    M4     16089      436223             3688
4    M5      8015      261749             3062
5    M6     14008      461368             3036
6    M8      4593      107773             4261
7    M9      7092      316832             2238
